# Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial - Inteligencia Artificial para Juegos (Game AI) - Proyecto: Búsqueda en el entorno Pacman

Fuente: [UC Berkeley CS188 Intro to AI](http://ai.berkeley.edu/search.html)

Instalación:


1.   Tener instalado Python. Activar versión 2.7. Si usa la plataforma [Anaconda](https://www.anaconda.com/), se recomienda crear un entorno virtual para el proyecto, como se muestra [aquí](https://unipython.com/entornos-virtuales-en-python-y-anaconda/). En dicho enlace, también hay instrucciones para instalar entornos virtuales en Python mismo.
2.   Descargar y descomprimir el [paquete](http://ai.berkeley.edu/projects/release/search/v1/001/search.zip) de archivos.
3. Desde un entorno con Python 2.7, donde se haya accedido a la ruta que contiene los archivos del proyecto, se pueden empezar a ejecutar los comandos desde la terminal.


## Comandos útiles
Ya están implementadas las funciones de BFS y DFS:

`python pacman.py -l mediumMaze -p SearchAgent -a fn=bfs`



Se le pide como **tarea** para la sesión 4, implementar la función de `aStarSearch (astar)` y `uniformCostSearch (ucs)`. Para verificar su funcionamiento, ejecutar el siguiente comando.



`python pacman.py -l mediumMaze -p SearchAgent -a fn=astar,heuristic=manhattanHeuristic`



Otros ejemplos de comandos:


```
python pacman.py
python pacman.py --layout testMaze --pacman GoWestAgent
python pacman.py --layout tinyMaze --pacman GoWestAgent
python pacman.py -h
python pacman.py -l tinyMaze -p SearchAgent -a fn=tinyMazeSearch
python pacman.py -l tinyMaze -p SearchAgent
python pacman.py -l mediumMaze -p SearchAgent
python pacman.py -l bigMaze -z .5 -p SearchAgent
python pacman.py -l mediumMaze -p SearchAgent -a fn=bfs
python pacman.py -l bigMaze -p SearchAgent -a fn=bfs -z .5
python eightpuzzle.py
python pacman.py -l mediumMaze -p SearchAgent -a fn=ucs
python pacman.py -l mediumDottedMaze -p StayEastSearchAgent
python pacman.py -l mediumScaryMaze -p StayWestSearchAgent
python pacman.py -l bigMaze -z .5 -p SearchAgent -a fn=astar,heuristic=manhattanHeuristic 
python pacman.py -l tinyCorners -p SearchAgent -a fn=bfs,prob=CornersProblem
python pacman.py -l mediumCorners -p SearchAgent -a fn=bfs,prob=CornersProblem
python pacman.py -l mediumCorners -p AStarCornersAgent -z 0.5
python pacman.py -l testSearch -p AStarFoodSearchAgent
python pacman.py -l trickySearch -p AStarFoodSearchAgent
python pacman.py -l bigSearch -p ClosestDotSearchAgent -z .5 
python pacman.py -l bigSearch -p ApproximateSearchAgent -z .5 -q
```




In [0]:
#@title search.py (Implementación de BFS y DFS realizadas: COMPLETAR FUNCIONES RESTANTES PARA LA SESIÓN 4)
# search.py
# ---------
# Licensing Information:  You are free to use or extend these projects for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to UC Berkeley, including a link to http://ai.berkeley.edu.
# 
# Attribution Information: The Pacman AI projects were developed at UC Berkeley.
# The core projects and autograders were primarily created by John DeNero
# (denero@cs.berkeley.edu) and Dan Klein (klein@cs.berkeley.edu).
# Student side autograding was added by Brad Miller, Nick Hay, and
# Pieter Abbeel (pabbeel@cs.berkeley.edu).


"""
In search.py, you will implement generic search algorithms which are called by
Pacman agents (in searchAgents.py).
"""

import util



class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Create a search tree Node, derived from a parent by an action."
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node %s>" % (self.state,)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        "List the nodes reachable in one step from this node."
        return [self.child_node(problem, action)
                for action in problem.getSuccessors(self.state)]

    def child_node(self, problem, action):
        "[Figure 3.10]"
        next = action[0]
        return Node(next, self, action[1], self.path_cost+action[2])

    def solution(self):
        "Return the sequence of actions to go from the root to this node."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Return a list of nodes forming the path from the root to this node."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)


class SearchProblem:
    """
    This class outlines the structure of a search problem, but doesn't implement
    any of the methods (in object-oriented terminology: an abstract class).

    You do not need to change anything in this class, ever.
    """

    def getStartState(self):
        """
        Returns the start state for the search problem.
        """
        util.raiseNotDefined()

    def isGoalState(self, state):
        """
          state: Search state

        Returns True if and only if the state is a valid goal state.
        """
        util.raiseNotDefined()

    def getSuccessors(self, state):
        """
          state: Search state

        For a given state, this should return a list of triples, (successor,
        action, stepCost), where 'successor' is a successor to the current
        state, 'action' is the action required to get there, and 'stepCost' is
        the incremental cost of expanding to that successor.
        """
        util.raiseNotDefined()

    def getCostOfActions(self, actions):
        """
         actions: A list of actions to take

        This method returns the total cost of a particular sequence of actions.
        The sequence must be composed of legal moves.
        """
        util.raiseNotDefined()



def tinyMazeSearch(problem):
    """
    Returns a sequence of moves that solves tinyMaze.  For any other maze, the
    sequence of moves will be incorrect, so only use this for tinyMaze.
    """
    from game import Directions
    s = Directions.SOUTH
    w = Directions.WEST
    return  [s, s, w, s, w, w, s, w]

def depthFirstSearch(problem):
    """
    Search the deepest nodes in the search tree first.

    Your search algorithm needs to return a list of actions that reaches the
    goal. Make sure to implement a graph search algorithm.

    To get started, you might want to try some of these simple commands to
    understand the search problem that is being passed in:

    print "Start:", problem.getStartState()
    print "Is the start a goal?", problem.isGoalState(problem.getStartState())
    print "Start's successors:", problem.getSuccessors(problem.getStartState())
    """
    "*** YOUR CODE HERE ***"
    #A node whose state is the initial state of the problem
    node = Node(problem.getStartState())
    frontier = util.Stack() #A LIFO list (stack) 
    frontier.push(node) #with node as the only element
    explored = set() #An empty set

    if problem.isGoalState(problem.getStartState()): return node.solution()
    
    while not frontier.isEmpty():
        node = frontier.pop()
        if problem.isGoalState(node.state): return node.solution()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored:
                frontier.push(child)
    return []



def breadthFirstSearch(problem):
    """Search the shallowest nodes in the search tree first."""
    "*** YOUR CODE HERE ***"

    #A node whose state is the initial state of the problem
    node = Node(problem.getStartState())
    frontier = util.Queue() #A FIFO list (queue) 
    frontier.push(node) #with node as the only element
    explored = set() #An empty set

    if problem.isGoalState(problem.getStartState()): return node.solution()
    while not frontier.isEmpty():
        node = frontier.pop()
        if problem.isGoalState(node.state): return node.solution()
        explored.add(node.state)
        for child in node.expand(problem):
            if (child.state not in explored) and (child not in frontier.list):
                frontier.push(child)
    return []


def uniformCostSearch(problem):
    """Search the node of least total cost first."""
    "*** YOUR CODE HERE ***"

    #Este es el inicio de la implementación sugerida
    #si desea, puede definir completamente esta función,
    #y no tomar en cuenta esta guía

    #A node whose state is the initial state of the problem
    node = Node(problem.getStartState())
    frontier = util.PriorityQueue() #A priority queue ordered by node.path_cost
    frontier.update(node, node.path_cost) #with node as the only element
    explored = set() #An empty set
    #Complete el código, de acuerdo al algoritmo visto en las diapositivas de teoría...


def nullHeuristic(state, problem=None):
    """
    A heuristic function estimates the cost from the current state to the nearest
    goal in the provided SearchProblem.  This heuristic is trivial.
    """
    return 0

def aStarSearch(problem, heuristic=nullHeuristic):
    """Search the node that has the lowest combined cost and heuristic first."""
    "*** YOUR CODE HERE ***"

    #Este es el inicio de la implementación sugerida
    #si desea, puede definir completamente esta función,
    #y no tomar en cuenta esta guía

    #A node whose state is the initial state of the problem
    node = Node(problem.getStartState())
    frontier = util.PriorityQueue() #A priority queue ordered by f=g+h
    frontier.update(node, ?) #with node as the only element
    explored = set() #An empty set
    
    #Complete el código, de acuerdo al algoritmo visto en las diapositivas de teoría...


# Abbreviations
bfs = breadthFirstSearch
dfs = depthFirstSearch
astar = aStarSearch
ucs = uniformCostSearch

## Tarea
Completar las funciones `uniformCostSearch` y `aStarSearch` para la sesión 4. Se habilitará una carpeta en el campus virtual para que suban sus archivos `search.py`. Solo se debe subir dicho fichero. Incluir en los comentarios del código sus nombres completos.